In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
import pandas as pd
import numpy as np
import torch
import datetime
from tqdm import tqdm_notebook
from torch import nn
from torchvision import transforms
from cnn_finetune import make_model

import model as M
import unet_parts
from dataset import SegmentationDataset, SegmentationInferenceDataset
from data import *
from util import *

In [3]:
torch.cuda.is_available()
device = torch.device('cuda')

# Setup dataset

In [4]:
train_df, val_df, _ = get_dfs()

In [5]:
val_df.shape

(800, 5)

In [6]:
train_dataset = SegmentationDataset(train_df, size=(128, 128))
val_dataset = SegmentationInferenceDataset(val_df, input_size=(128, 128), with_gt=True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)

# Training

In [7]:
gc.collect()
torch.cuda.empty_cache()

model = M.UNetRes(1, n_classes=1)
#model = M.UNet(1, n_classes=1)
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

In [10]:
def train(model, n_epoch, train_iter, val_iter):
    for epoch in range(n_epoch):
        model.train()
        
        gc.collect()
        torch.cuda.empty_cache()
        
        total_loss = 0
        total_size = 0
        
        for batch_idx, (data, target) in enumerate(train_iter):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            
            # Forward
            output = model(data)
            loss = criterion(output.view(-1), target.view(-1))
            
            total_loss += loss.item()
            total_size += data.size(0)
            
            # Backward
            loss.backward()
            optimizer.step()
            
            if batch_idx % 100 == 0:
                now = datetime.datetime.now()
                print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {:.6f}'.format(
                    now,
                    epoch, batch_idx * len(data), len(train_iter.dataset),
                    100. * batch_idx / len(train_iter), total_loss / total_size))
                
        gc.collect()
        torch.cuda.empty_cache()
        
        with torch.no_grad():
            iou = evaluate(model, val_iter, device=device)
        print('[{}] Train Epoch: {}\tIoU: {:.6f}'.format(now, epoch, iou))
    return model

In [11]:
model = train(model, 20, train_loader, val_loader)

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


[2018-10-02 19:12:07.581079] Train Epoch: 0 [0/3200 (0%)]	Average loss: 0.044788
[2018-10-02 19:12:36.673081] Train Epoch: 0 [1600/3200 (50%)]	Average loss: 0.028662
[2018-10-02 19:12:36.673081] Train Epoch: 0	IoU: 0.514250
[2018-10-02 19:13:20.284894] Train Epoch: 1 [0/3200 (0%)]	Average loss: 0.030981
[2018-10-02 19:13:49.415707] Train Epoch: 1 [1600/3200 (50%)]	Average loss: 0.024259
[2018-10-02 19:13:49.415707] Train Epoch: 1	IoU: 0.507500
[2018-10-02 19:14:25.617082] Train Epoch: 2 [0/3200 (0%)]	Average loss: 0.027542
[2018-10-02 19:14:54.731880] Train Epoch: 2 [1600/3200 (50%)]	Average loss: 0.023317
[2018-10-02 19:14:54.731880] Train Epoch: 2	IoU: 0.440250
[2018-10-02 19:15:30.918308] Train Epoch: 3 [0/3200 (0%)]	Average loss: 0.013193
[2018-10-02 19:16:00.027250] Train Epoch: 3 [1600/3200 (50%)]	Average loss: 0.021636
[2018-10-02 19:16:00.027250] Train Epoch: 3	IoU: 0.502250
[2018-10-02 19:16:36.166392] Train Epoch: 4 [0/3200 (0%)]	Average loss: 0.013633
[2018-10-02 19:17:05.32

In [12]:
save_model(model, 'unet_resblock')

In [47]:
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        y = model(data)
        break
y.shape

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


torch.Size([16, 1, 128, 128])

In [21]:
transforms.ToPILImage?

In [8]:
x = torch.Tensor(np.random.rand(4, 1, 128, 128))

x1 = unet_parts.inconv(1, 64)(x)
x2 = unet_parts.double_conv(64, 128)(x1)
x2.shape

torch.Size([4, 128, 128, 128])

In [9]:
x = torch.Tensor(np.random.rand(4, 1, 128, 128))

x1 = unet_parts.inconv(1, 64)(x)
x2 = unet_parts.down(64, 128)(x1)
x2.shape

torch.Size([4, 128, 64, 64])

In [13]:
x = torch.Tensor(np.random.rand(4, 1, 128, 128))

x1 = unet_parts.inconv(1, 64)(x)
x2 = unet_parts.BasicBlock(64, 128)(x1)
x2.shape

torch.Size([4, 128, 128, 128])